In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import random
from sklearn.metrics import confusion_matrix, mean_squared_error, log_loss, accuracy_score
import time
from tensorflow.keras.models import model_from_json
from tensorflow.keras import layers
from tensorflow.keras import optimizers

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
Xdev = np.load('encoded_data_clip_fast/Xdev.npy')
ydev = np.load('encoded_data_clip_fast/ydev.npy')

In [5]:
Xdev.shape

(13863, 446)

In [10]:
# pred_dev = loaded_model.predict(Xdev)
# dev_mse = mean_squared_error(ydev, pred_dev)
dev_mse = np.load('results_clip_fast/dev_pred_mse.npy')
dev_mse

array(0.03386871)

## Use permutation to analyze the importance of features

In [9]:
# n_example, n_features = Xdev.shape
# n_features

## Complete running the code within .py file and load the results

In [2]:
## train within the .py file. Now load the data
permutsample = np.load('results_clip_fast/permut_dev_results.npy')
permutsample.shape

(10, 446)

In [16]:
n_sample, n_features = permutsample.shape
# print mean and standard deviation of error
errperformance = np.zeros((n_features, 2))
for i in range(n_features):
    errperformance[i,0] = np.mean(permutsample[:,i])
    errperformance[i,1] = np.std(permutsample[:,i]) / np.sqrt(n_sample)
errperformance[:,0] = errperformance[:,0] - dev_mse

print(errperformance)

[[ 3.27303356e-03  5.00927573e-05]
 [ 3.14807061e-03  2.09584702e-05]
 [ 1.41817719e-03  2.79988316e-05]
 [ 3.61142029e-03  3.84729787e-05]
 [ 1.53004306e-03  2.98738924e-05]
 [ 1.97724803e-02  1.30872053e-04]
 [ 4.65335143e-05  3.06386952e-06]
 [ 5.22384255e-04  1.53555710e-05]
 [ 1.41829312e-03  2.13532721e-05]
 [ 3.63287830e-05  4.60812796e-06]
 [ 3.63225343e-04  9.72806957e-06]
 [ 1.49586180e-03  1.25116969e-05]
 [ 1.96471395e-02  5.80586103e-05]
 [ 1.40738666e-03  1.58592134e-05]
 [ 1.55916308e-03  2.54972700e-05]
 [ 9.14413827e-04  1.43264131e-05]
 [ 2.77238896e-03  3.78504821e-05]
 [ 1.00726071e-03  1.70037740e-05]
 [ 1.14313067e-02  6.80896129e-05]
 [ 8.11023785e-04  8.83953004e-06]
 [ 1.81960233e-05  1.90502423e-06]
 [ 5.24068066e-05  3.92499871e-06]
 [ 3.18142638e-06  7.05843722e-07]
 [ 3.29878827e-05  2.69414973e-06]
 [ 4.85380778e-06  1.54732735e-06]
 [ 2.96544877e-06  1.51198759e-06]
 [ 3.68872572e-04  1.36860554e-05]
 [ 3.03003451e-04  9.05878093e-06]
 [ 3.04819397e-06  2

In [17]:
features = pd.read_csv('encoded_data_clip_fast/encoded_columns_name.txt', delimiter='\t', header=None)

In [18]:
features.columns = ['feature_name']

In [19]:
features['delta_mse'] = errperformance[:,0]
features['std_permut_mse'] = errperformance[:,1]
features = features.sort_values('delta_mse', axis=0, ascending=False)
features.reset_index(drop=True)

,feature_name,delta_mse,std_permut_mse
0,PaidCount,1.977248e-02,1.308721e-04
1,EXOEnabledUsers,1.964714e-02,5.805861e-05
2,PPDEnabledUsers,1.143131e-02,6.808961e-05
3,PaidEXOSeats,1.078308e-02,5.760954e-05
4,CountryCode=US,7.900167e-03,4.175647e-05
5,PaidProplusSeats,6.791015e-03,4.978124e-05
6,Languange=en,5.127096e-03,4.667751e-05
7,PaidOD4BSeats,3.886245e-03,3.084944e-05
8,SfBSubscriptionsCount,3.611420e-03,3.847298e-05
9,TotalUsers,3.556611e-03,4.022759e-05


In [19]:
list(features.head(40)['feature_name'])

['PaidCount',
 'EXOEnabledUsers',
 'PPDEnabledUsers',
 'PaidEXOSeats',
 'CountryCode=US',
 'PaidProplusSeats',
 'Languange=en',
 'PaidOD4BSeats',
 'SfBSubscriptionsCount',
 'TotalUsers',
 'AllupSeats',
 'EXOSubscriptionsCount',
 'TeamEnabledUsers',
 'CountryCode=GB',
 'HasEXO',
 'PaidKaizalaSeats',
 'FirstPaidEXOStartDate',
 'FirstPaidOD4BStartDate',
 'TopParents_Industry=0',
 'CountryCode=DE',
 'O365EnabledUsers',
 'FirstPaidProPlusStartDate',
 'CountryCode=NL',
 'OD4BEnabledUsers',
 'TeamsSubscriptionsCount',
 'NonTrialSubscriptionsCount',
 'PaidSPOSeats',
 'ActivatedSubscriptionTotalCount',
 'OD4BSubscriptionsCount',
 'SPOEnabledUsers',
 'CountryCode=AU',
 'PaidSFBSeats',
 'Languange=de',
 'HasSPO',
 'CountryCode=CA',
 'Languange=es',
 'HasOD4B',
 'M365EnabledUsers',
 'YammerEnabledUsers',
 'CountryCode=FR']

In [17]:
[s for s in list(features['feature_name']) if 'TopParents_Industry' in s]

['TopParents_Industry=0',
 'TopParents_Industry=Partner Professional Services',
 'TopParents_Industry=Smart Spaces',
 'TopParents_Industry=Consumer Goods',
 'TopParents_Industry=Professional Services',
 'TopParents_Industry=Health Provider',
 'TopParents_Industry=Retailers',
 'TopParents_Industry=Nonprofit',
 'TopParents_Industry=Other - Unsegmented',
 'TopParents_Industry=Discrete Manufacturing',
 'TopParents_Industry=Media & Entertainment',
 'TopParents_Industry=Chemicals & Agrochemicals',
 'TopParents_Industry=National Government',
 'TopParents_Industry=Telecommunications',
 'TopParents_Industry=Banking & Capital Markets',
 'TopParents_Industry=Primary & Secondary Edu/K-12',
 'TopParents_Industry=Travel & Transportation',
 'TopParents_Industry=Insurance',
 'TopParents_Industry=Higher Education',
 'TopParents_Industry=Energy',
 'TopParents_Industry=Automotive',
 'TopParents_Industry=Local Regional Government',
 'TopParents_Industry=Health Payor',
 'TopParents_Industry=Pharmaceuticals

In [132]:
# a = np.arange(12).reshape((3,4))
# a

In [133]:
# a[:,1] = np.random.permutation(a[:,1])

In [134]:
# a = np.arange(12).reshape((3,4))
# np.random.permutation(a[:,1])
# a

In [135]:
# len(a[0])

In [136]:
# a = np.arange(12).reshape((3,4))

# for i in range(len(a[0])):
#     permut_a = np.zeros(np.shape(a))
#     permut_a[:] = a
#     permut_a[:,i] = np.random.permutation(a[:,i])
#     print(permut_a)

## trial with fake settings

In [105]:
t1 = time.time()

permut_X = Xdev

permut_X[:,990] = np.random.permutation(Xdev[:,990])
pred_permute = loaded_model.predict(permut_X)
mse_permute = mean_squared_error(ydev, pred_permute)
print(mse_permute)

t2 = time.time()
print('used time in {} seconds'.format(t2-t1))

0.08164757472081949
used time in 0.2709469795227051 seconds


In [8]:
randsample = 3
n_features = 5

In [110]:
permutsample = np.zeros((randsample, n_features))
for trying in range(randsample):
    t1 = time.time()
    print('This is sample {}'.format(trying))
    for i in range(n_features):
        permut_X = np.zeros(np.shape(Xdev))
        permut_X[:] = Xdev
        permut_X[:,-i] = np.random.permutation(Xdev[:,-i])
        permut_pred = loaded_model.predict(permut_X)
        permut_mse = mean_squared_error(ydev, permut_pred)
        permutsample[trying, -i] = permut_mse
    t2 = time.time()
    print('Use time in {} seconds'.format(t2-t1))
    print('*' * 20)


This is sample 0
Use time in 1.5444622039794922 seconds
********************
This is sample 1
Use time in 1.551302194595337 seconds
********************
This is sample 2
Use time in 1.545614242553711 seconds
********************


In [111]:
print(permutsample)

[[0.08166467 0.07619916 0.09034454 0.13429679 0.08338546]
 [0.08168226 0.08278596 0.08930428 0.13448791 0.08589154]
 [0.08167277 0.08031207 0.088329   0.17884274 0.08598124]]


In [112]:
# np.save('results/permut_dev_results.npy', permutsample)

In [14]:
len(features)

489

In [3]:
feature_selected = features.loc[features['delta_mse'] >= 10e-04]

NameError: name 'features' is not defined

In [2]:
10e-02

0.1

## Look at some examples on dev set

In [15]:
outputs_name = [
     'AR_exchange_06',
 'AR_sharepoint_06',
 'AR_skype_06',
 'AR_teams_06',
 'AR_od4b_06',
 'AR_onenote_06',
 'AR_word_06',
 'AR_excel_06',
 'AR_powerpoint_06',
 'AR_outlook_06',
 'AR_eslt_06',
 'AR_officeclient_06'
]

In [16]:
Xdev_show = Xdev[:3]
ydev_show = ydev[:3]
predictions = loaded_model.predict(Xdev_show)
# print('predictions shape:', predictions.shape)
data = np.concatenate((ydev_show,predictions), axis=0)
data_df = pd.DataFrame(data=data, columns=outputs_name)
data_df

,AR_exchange_06,AR_sharepoint_06,AR_skype_06,AR_teams_06,AR_od4b_06,AR_onenote_06,AR_word_06,AR_excel_06,AR_powerpoint_06,AR_outlook_06,AR_eslt_06,AR_officeclient_06
0,0.734375,0.000000,0.000000,0.000000,0.000000,0.000000,0.047222,0.030556,0.002778,0.297222,0.652778,0.297222
1,0.887500,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.012500,0.000000,0.062500,0.887500,0.075000
2,1.375000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.375000,0.000000
3,0.736666,0.000647,-0.001815,0.017586,0.018733,0.006842,0.015055,0.009503,0.009471,0.283905,0.735242,0.278910
4,0.756156,-0.000063,-0.005246,0.012503,0.066448,0.007137,-0.008417,-0.009617,0.002480,0.066241,0.762578,0.066907
5,1.396505,0.013625,0.003225,-0.001619,-0.017340,-0.002784,0.009152,-0.039731,-0.013593,0.036768,1.303271,0.015036


In [17]:
for i in range(len(ydev_show)):
    print('This is for dev example {}'.format(i))
    
    n = Xdev_show.shape[1]
    predictions = loaded_model.predict(Xdev_show[i].reshape((1, n)))
    
    for j in range(len(ydev_show[i])):
        print('{}_true: {}'.format(outputs_name[j], ydev_show[i][j]))
        print('{}_pred: {}'.format(outputs_name[j], predictions[0][j]))
        print('='*50)
    
    print('*'*20)

This is for dev example 0
AR_exchange_06_true: 0.734375
AR_exchange_06_pred: 0.7366665005683899
AR_sharepoint_06_true: 0.0
AR_sharepoint_06_pred: 0.0006472542881965637
AR_skype_06_true: 0.0
AR_skype_06_pred: -0.0018151979893445969
AR_teams_06_true: 0.0
AR_teams_06_pred: 0.01758648082613945
AR_od4b_06_true: 0.0
AR_od4b_06_pred: 0.018733076751232147
AR_onenote_06_true: 0.0
AR_onenote_06_pred: 0.006842270493507385
AR_word_06_true: 0.04722222222222222
AR_word_06_pred: 0.0150546133518219
AR_excel_06_true: 0.03055555555555556
AR_excel_06_pred: 0.009503096342086792
AR_powerpoint_06_true: 0.0027777777777777783
AR_powerpoint_06_pred: 0.009470563381910324
AR_outlook_06_true: 0.2972222222222222
AR_outlook_06_pred: 0.2839052677154541
AR_eslt_06_true: 0.6527777777777778
AR_eslt_06_pred: 0.7352428436279297
AR_officeclient_06_true: 0.2972222222222222
AR_officeclient_06_pred: 0.27891072630882263
********************
This is for dev example 1
AR_exchange_06_true: 0.8875
AR_exchange_06_pred: 0.756155908